importing our libs (only numpy for linear algebra and arrays)

In [1]:
import numpy as np
import os
import requests
import zipfile


### --- prepare our words from glove file ---

In [2]:
glove_url = "https://ftp.cdc.gov/pub/software/RegistryPlus/CLEW/glove.6B.50d.txt"
glove_file = "glove.6B.50d.txt"

if not os.path.exists(glove_file):
    print("downloading glove embeddings...")
    r = requests.get(glove_url, stream=True)
    with open(glove_file, "wb") as f:
        for chunk in r.iter_content(chunk_size=64192):
            f.write(chunk)
    print("glove embeddings ready.")


downloading glove embeddings...
glove embeddings ready.


### --- load data, and made a hashmap to save the vector of every word ---

In [3]:
EMB = {}
LIMIT = 9999999999
done = False



with open('glove.6B.50d.txt', "r", encoding="utf8") as f:
    i = 0
    while i < LIMIT:
        line = f.readline()  # line -> `word float1 float2 float3 ... float50`
        if line == "":
            break # that means we reached the end
        words = line.split()
        if (len(words)) < 2: # this check is cuz there is no need to seg fault obviously..
            continue
        word = words[0]
        EMB[word] = [float(x) for x in words[1:]] # start from 1 to skip the actual word
        i += 1

### --- tokenize the sentence and make it a unidimentional vector with the mean of data ---

In [4]:
def tokenize_sentence(text: str):
    toks = text.lower().replace(".", "").replace(",", "").split() # tokenize
    vecs = []

    for i in range(len(toks)):
        if (toks[i] in EMB):
            vecs.append(EMB[toks[i]])
    if len(vecs) == 0:
        return np.zeros(50) # 50 zeros if we didnt figure out the sentence at all
    mean = np.mean(vecs, axis=0)
    return mean

### --- we will check the diff  between the 2 sentences' vectors. ---

In [5]:

def calculate_cosin_of_2_sentences(s1_vecs : list[float], s2_vecs: list[float]):
    norm_s1 = np.linalg.norm(s1_vecs)
    norm_s2 = np.linalg.norm(s2_vecs)

    print(norm_s1, norm_s2)
    if norm_s1 == 0 and norm_s2 == 0:
        return 0
    dot_s1_s2 = np.dot(s1_vecs, s2_vecs)
    cosin = dot_s1_s2 / (norm_s1 * norm_s2)
    return cosin

### --- run it . ---

In [6]:
def sentences_diff_perc(s1: str, s2: str):
    vecs1 = tokenize_sentence(s1)
    vecs2 = tokenize_sentence(s2)

    return calculate_cosin_of_2_sentences(vecs1, vecs2)

s1 = "i love dogs"

s2 = "i love cats"


s3 = "lets go"

s4 = "the light bump is near us"


s5 = "bananas are yellow"

s6 = "i love cats"

perc_of_assemblence1 = sentences_diff_perc(s1, s2)
print(f"perc of -> {s1} and {s2} -> {perc_of_assemblence1}")
perc_of_assemblence2 = sentences_diff_perc(s1, s3)
print(f"perc of -> {s1} and {s3} -> {perc_of_assemblence2}")
perc_of_assemblence3 = sentences_diff_perc(s3, s4)
print(f"perc of -> {s2} and {s3} -> {perc_of_assemblence3}")



perc_of_assemblence4 = sentences_diff_perc(s5, s6)
print(f"perc of -> {s5} and {s6} -> {perc_of_assemblence4}")

4.643116175134064 4.490997932768985
perc of -> i love dogs and i love cats -> 0.982958612446962
4.643116175134064 4.287713560441608
perc of -> i love dogs and lets go -> 0.7667008860955261
4.287713560441608 3.814651065325064
perc of -> i love cats and lets go -> 0.6693422494123608
4.042957153294079 4.490997932768985
perc of -> bananas are yellow and i love cats -> 0.5882994784359508


### --- plain simple. now we can see how close a sentence is to an other ! ----